In [ ]:
# Install required packages (run this first if not already installed)
import os
import subprocess
import sys


def install_packages():
    packages = [
        "llm>=0.17.0",
        "llm-anthropic>=0.3.0", 
        "llm-openai-plugin>=0.2.0",
        "llm-gemini>=0.2.0",
        "sqlite-utils>=3.37.0",
        "datasette>=1.0.0"
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ Installed {package}")
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to install {package}: {e}")

# Uncomment to install packages
# install_packages()

print("✓ Setup complete - Simon's LLM ecosystem ready!")


In [ ]:
# Import the LlamaAgent Simon ecosystem integration
import json

# Import our Simon ecosystem integration
try:
    from src.llamaagent.integration.simon_tools import SimonToolRegistry, create_simon_tools
    from src.llamaagent.llm.simon_ecosystem import LLMProvider, LLMTool, SimonEcosystemConfig, SimonLLMEcosystem
    print("✓ Successfully imported LlamaAgent Simon ecosystem")
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("Make sure you're running from the project root directory")

# Configure API keys (set these in your environment)
config = SimonEcosystemConfig(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    gemini_api_key=os.getenv("GEMINI_API_KEY"),
    database_path="simon_ecosystem_demo.db",
    enabled_tools=[
        LLMTool.SQLITE,
        LLMTool.PYTHON, 
        LLMTool.COMMAND,
        LLMTool.QUICKJS
    ],
    log_conversations=True
)

print("✓ Configuration ready")
print(f"  - Database: {config.database_path}")
print(f"  - Enabled tools: {[tool.value for tool in config.enabled_tools]}")
print(f"  - OpenAI API key: {'✓' if config.openai_api_key else '✗'}")
print(f"  - Anthropic API key: {'✓' if config.anthropic_api_key else '✗'}")


In [ ]:
# Initialize the Simon ecosystem
ecosystem = SimonLLMEcosystem(config)

async def basic_chat_example():
    """Demonstrate basic chat functionality."""
    
    # Simple chat
    print("=== Basic Chat Example ===")
    
    try:
        response = await ecosystem.chat(
            prompt="Explain quantum computing in exactly 2 sentences.",
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=100
        )
        
        print("Prompt: Explain quantum computing in exactly 2 sentences.")
        print(f"Response: {response}")
        print()
        
    except Exception as e:
        print(f"Error: {e}")
        print("Note: This requires an OpenAI API key to be set")
    
    # Check ecosystem health
    print("=== Ecosystem Health Check ===")
    health = await ecosystem.health_check()
    print(json.dumps(health, indent=2))

# Run the example
await basic_chat_example()


In [ ]:
# Embedding generation example
async def embedding_example():
    """Demonstrate embedding generation."""
    
    print("=== Embedding Generation Example ===")
    
    try:
        # Generate embeddings for multiple texts
        texts = [
            "Machine learning is transforming technology",
            "Artificial intelligence enables smart systems", 
            "Quantum computing promises exponential speedups"
        ]
        
        embeddings = await ecosystem.embed(
            text=texts,
            model="text-embedding-3-small"
        )
        
        print(f"Generated embeddings for {len(texts)} texts")
        for i, (text, embedding) in enumerate(zip(texts, embeddings, strict=False)):
            print(f"Text {i+1}: {text}")
            print(f"Embedding dimensions: {len(embedding)}")
            print(f"First 5 values: {embedding[:5]}")
            print()
            
    except Exception as e:
        print(f"Error: {e}")
        print("Note: This requires an OpenAI API key to be set")

# Run embedding example
await embedding_example()


In [ ]:
async def multi_provider_example():
    """Demonstrate using multiple LLM providers."""
    
    print("=== Multi-Provider Chat Example ===")
    
    # Define the same question for all providers
    question = "What is the key benefit of using async/await in Python?"
    
    # List of models to try (some may not be available without API keys)
    models_to_try = [
        ("gpt-4o-mini", LLMProvider.OPENAI),
        ("claude-3-haiku", LLMProvider.ANTHROPIC),
        ("gemini-pro", LLMProvider.GEMINI),
        ("llama3.2", LLMProvider.OLLAMA),  # If running locally
    ]
    
    results = {}
    
    for model, provider in models_to_try:
        print(f"\n--- Testing {provider.value}: {model} ---")
        
        try:
            response = await ecosystem.chat(
                prompt=question,
                model=model,
                provider=provider,
                temperature=0.3,
                max_tokens=150
            )
            
            results[f"{provider.value}:{model}"] = {
                "success": True,
                "response": response,
                "length": len(response)
            }
            
            print(f"✓ Success ({len(response)} chars)")
            print(f"Response: {response[:100]}...")
            
        except Exception as e:
            results[f"{provider.value}:{model}"] = {
                "success": False,
                "error": str(e)
            }
            print(f"✗ Failed: {e}")
    
    # Summary
    print("\n=== Summary ===")
    successful = sum(1 for r in results.values() if r["success"])
    total = len(results)
    print(f"Successful providers: {successful}/{total}")
    
    return results

# Run multi-provider example
provider_results = await multi_provider_example()


In [ ]:
async def database_operations_example():
    """Demonstrate SQLite database operations."""
    
    print("=== Database Operations Example ===")
    
    try:
        # Create a sample table
        await ecosystem.use_tool(
            "sqlite",
            "create_table",
            table="ai_models",
            schema={
                "id": int,
                "name": str,
                "provider": str,
                "parameters": str,
                "performance_score": float
            }
        )
        print("✓ Created 'ai_models' table")
        
        # Insert sample data
        sample_models = [
            {
                "id": 1,
                "name": "gpt-4o",
                "provider": "OpenAI", 
                "parameters": "175B",
                "performance_score": 9.2
            },
            {
                "id": 2,
                "name": "claude-3-opus",
                "provider": "Anthropic",
                "parameters": "Unknown",
                "performance_score": 9.1
            },
            {
                "id": 3,
                "name": "gemini-pro",
                "provider": "Google",
                "parameters": "Unknown", 
                "performance_score": 8.8
            }
        ]
        
        for model in sample_models:
            await ecosystem.use_tool(
                "sqlite",
                "insert",
                table="ai_models",
                data=model
            )
        
        print(f"✓ Inserted {len(sample_models)} model records")
        
        # Query the data
        results = await ecosystem.use_tool(
            "sqlite",
            "query",
            sql="SELECT * FROM ai_models ORDER BY performance_score DESC"
        )
        
        print("\n--- Top AI Models by Performance ---")
        for row in results:
            print(f"{row['name']} ({row['provider']}): {row['performance_score']}")
        
        # Aggregate query
        avg_result = await ecosystem.use_tool(
            "sqlite", 
            "query",
            sql="SELECT AVG(performance_score) as avg_score, COUNT(*) as total_models FROM ai_models"
        )
        
        print("\n--- Statistics ---")
        stats = avg_result[0]
        print(f"Total models: {stats['total_models']}")
        print(f"Average performance: {stats['avg_score']:.2f}")
        
        return results
        
    except Exception as e:
        print(f"Database error: {e}")
        return []

# Run database example
db_results = await database_operations_example()


In [ ]:
async def code_execution_example():
    """Demonstrate code execution capabilities."""
    
    print("=== Code Execution Example ===")
    
    # Python code execution
    print("\n--- Python Code Execution ---")
    python_code = '''
# Calculate fibonacci sequence
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Generate first 10 fibonacci numbers
fib_sequence = [fibonacci(i) for i in range(10)]
print("Fibonacci sequence:", fib_sequence)

# Calculate some statistics
total = sum(fib_sequence)
average = total / len(fib_sequence)
print(f"Sum: {total}, Average: {average:.2f}")
'''
    
    try:
        result = await ecosystem.use_tool(
            "python",
            "run", 
            code=python_code
        )
        
        if result["success"]:
            print("✓ Python execution successful")
            print("Namespace keys:", list(result["namespace"].keys()))
        else:
            print(f"✗ Python execution failed: {result['error']}")
            
    except Exception as e:
        print(f"Python execution error: {e}")
    
    # JavaScript code execution (if available)
    print("\n--- JavaScript Code Execution ---") 
    js_code = '''
// Simple data processing
const data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const squared = data.map(x => x * x);
const sum = squared.reduce((a, b) => a + b, 0);

console.log("Original data:", data);
console.log("Squared values:", squared);  
console.log("Sum of squares:", sum);
console.log("Average:", sum / squared.length);
'''
    
    try:
        result = await ecosystem.use_tool(
            "quickjs",
            "run",
            code=js_code
        )
        
        if result["success"]:
            print("✓ JavaScript execution successful")
            print("Output:", result["output"])
        else:
            print(f"✗ JavaScript execution failed: {result['error']}")
            
    except Exception as e:
        print(f"JavaScript execution error: {e}")
    
    # System command execution
    print("\n--- System Command Execution ---")
    try:
        result = await ecosystem.use_tool(
            "command",
            "run",
            command="echo 'Hello from system command!' && date && python --version"
        )
        
        if result["success"]:
            print("✓ Command execution successful")
            print("Output:", result["stdout"])
        else:
            print(f"✗ Command failed: {result['stderr']}")
            
    except Exception as e:
        print(f"Command execution error: {e}")

# Run code execution examples
await code_execution_example()


In [ ]:
async def analytics_example():
    """Demonstrate conversation analytics."""
    
    print("=== Conversation Analytics Example ===")
    
    # Generate some sample conversations for analysis
    sample_prompts = [
        "What is machine learning?",
        "Explain neural networks",
        "How does reinforcement learning work?", 
        "What are transformers in AI?",
        "Describe computer vision applications"
    ]
    
    print("Generating sample conversations...")
    for i, prompt in enumerate(sample_prompts):
        try:
            # This will log conversations automatically
            response = await ecosystem.chat(
                prompt=prompt,
                model="gpt-4o-mini",
                temperature=0.5
            )
            print(f"✓ Conversation {i+1} logged")
        except Exception as e:
            print(f"✗ Failed to generate conversation {i+1}: {e}")
    
    # Get conversation statistics  
    print("\n--- Conversation Statistics ---")
    try:
        stats = await ecosystem.get_conversation_stats()
        
        print(f"Total conversations: {stats.get('total_conversations', 0)}")
        print(f"Unique providers: {stats.get('unique_providers', 0)}")
        print(f"Unique models: {stats.get('unique_models', 0)}")
        print(f"Total tokens: {stats.get('total_tokens', 0)}")
        print(f"Total cost: ${stats.get('total_cost', 0):.4f}")
        print(f"Average cost per conversation: ${stats.get('avg_cost_per_conversation', 0):.4f}")
        
    except Exception as e:
        print(f"Stats error: {e}")
    
    # Search conversations
    print("\n--- Conversation Search ---")
    try:
        search_results = await ecosystem.search_conversations(
            query="learning",
            limit=3
        )
        
        print(f"Found {len(search_results)} conversations containing 'learning':")
        for i, conv in enumerate(search_results):
            print(f"{i+1}. {conv['prompt'][:50]}...")
            print(f"   Response: {conv['response'][:80]}...")
            print()
            
    except Exception as e:
        print(f"Search error: {e}")
    
    return stats

# Run analytics example
analytics_stats = await analytics_example()


In [ ]:
# Create Simon's tool registry for LlamaAgent integration
simon_tools = create_simon_tools(config)
tool_registry = simon_tools.get_registry()

async def llamaagent_integration_example():
    """Demonstrate LlamaAgent integration with Simon's tools."""
    
    print("=== LlamaAgent Integration with Simon's Tools ===")
    
    # List available tools
    available_tools = tool_registry.list_tools()
    print(f"Available Simon tools: {len(available_tools)}")
    for tool_name in available_tools:
        tool = tool_registry.get_tool(tool_name)
        print(f"  - {tool_name}: {tool.description}")
    
    print("\n--- Using Individual Tools ---")
    
    # Use LLM chat tool
    llm_chat_tool = tool_registry.get_tool("llm_chat")
    if llm_chat_tool:
        result = await llm_chat_tool.execute(
            prompt="What are the key benefits of using SQLite for data storage?",
            model="gpt-4o-mini"
        )
        
        if result["success"]:
            print("✓ LLM Chat Tool:")
            print(f"  Response: {result['response'][:100]}...")
        else:
            print(f"✗ LLM Chat failed: {result['error']}")
    
    # Use SQLite tool
    sqlite_tool = tool_registry.get_tool("sqlite_query")
    if sqlite_tool:
        result = await sqlite_tool.execute(
            sql="SELECT COUNT(*) as conversation_count FROM conversations"
        )
        
        if result["success"]:
            print("✓ SQLite Query Tool:")
            print(f"  Query result: {result['data']}")
        else:
            print(f"✗ SQLite query failed: {result['error']}")
    
    # Use ecosystem stats tool
    stats_tool = tool_registry.get_tool("ecosystem_stats")
    if stats_tool:
        result = await stats_tool.execute()
        
        if result["success"]:
            print("✓ Ecosystem Stats Tool:")
            print(f"  Health status: {result['health_status']['status']}")
            print(f"  Database status: {result['health_status']['database']}")
        else:
            print(f"✗ Stats failed: {result['error']}")
    
    # Health check for all tools
    print("\n--- Tool Health Check ---")
    health = await simon_tools.health_check()
    print(f"Overall ecosystem health: {health['status']}")
    
    return tool_registry

# Run integration example
integrated_tools = await llamaagent_integration_example()


In [ ]:
async def data_export_example():
    """Demonstrate data export capabilities."""
    
    print("=== Data Export Example ===")
    
    # Export conversations to JSON
    try:
        json_export = await ecosystem.export_data(
            table="conversations",
            format="json",
            output_path="conversations_export.json"
        )
        
        print(f"✓ Exported conversations to: {json_export}")
        
        # Check file size
        if os.path.exists(json_export):
            file_size = os.path.getsize(json_export)
            print(f"  File size: {file_size} bytes")
            
            # Show first few lines
            with open(json_export, 'r') as f:
                content = f.read()
                if content:
                    data = json.loads(content)
                    print(f"  Records exported: {len(data)}")
                    if data:
                        print(f"  Sample record keys: {list(data[0].keys())}")
                else:
                    print("  No data exported")
        
    except Exception as e:
        print(f"JSON export error: {e}")
    
    # Export to CSV format
    try:
        csv_export = await ecosystem.export_data(
            table="conversations", 
            format="csv",
            output_path="conversations_export.csv"
        )
        
        print(f"✓ Exported conversations to: {csv_export}")
        
        if os.path.exists(csv_export):
            file_size = os.path.getsize(csv_export)
            print(f"  CSV file size: {file_size} bytes")
            
    except Exception as e:
        print(f"CSV export error: {e}")
    
    # Datasette server demo (commented out as it starts a server)
    print("\n--- Datasette Server Integration ---")
    print("To start a Datasette server for data exploration:")
    print(f"  datasette {config.database_path} --port 8001")
    print("  Then visit: http://localhost:8001")
    
    return json_export

# Run export example
export_path = await data_export_example()


In [ ]:
# Production deployment configuration example
production_config = SimonEcosystemConfig(
    # Security: Use environment variables for API keys
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    gemini_api_key=os.getenv("GEMINI_API_KEY"),
    
    # Database: Use persistent storage
    database_path=os.getenv("LLM_DATABASE_PATH", "/app/data/llm_ecosystem.db"),
    
    # Performance: Optimize for production
    default_chat_model="gpt-4o-mini",  # Cost-effective default
    default_embedding_model="text-embedding-3-small",
    
    # Tools: Enable production-safe tools
    enabled_tools=[
        LLMTool.SQLITE,
        LLMTool.PYTHON,  # Sandboxed execution
        LLMTool.QUICKJS,  # JavaScript execution
        # LLMTool.DOCKER,  # Enable if Docker is available
        # LLMTool.COMMAND,  # Disable in production for security
    ],
    
    # Monitoring: Enable comprehensive logging
    log_conversations=True,
    log_level="INFO",
    
    # Datasette: Configure for monitoring
    datasette_port=8001,
    
    # Docker: Configure for safe execution
    docker_image="python:3.11-slim",
    docker_timeout=60  # Shorter timeout for production
)

print("=== Production Configuration ===")
print(f"Database path: {production_config.database_path}")
print(f"Enabled tools: {[tool.value for tool in production_config.enabled_tools]}")
print(f"Default chat model: {production_config.default_chat_model}")
print(f"Logging enabled: {production_config.log_conversations}")

# Production deployment checklist
deployment_checklist = [
    "✓ Set API keys in environment variables",
    "✓ Configure persistent database storage", 
    "✓ Enable conversation logging",
    "✓ Set up monitoring with Datasette",
    "✓ Configure appropriate tool permissions",
    "✓ Set resource limits and timeouts",
    "✓ Enable backup and recovery procedures",
    "✓ Set up health monitoring endpoints",
    "✓ Configure rate limiting",
    "✓ Test all provider integrations"
]

print("\n=== Production Deployment Checklist ===")
for item in deployment_checklist:
    print(item)
